# Qredo tokenomics simulation - parameter sweep

### June 2023

## 1. Imports

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
code_dir = os.path.realpath(os.path.join(os.getcwd(), ".."))

import sys
sys.path.append(code_dir)

from mechaqredo.sim import run_param_sweep_sim

## 2. Load scenarios

In [3]:
forecast_length = 365*2
outfolder = os.path.realpath("../data/sim_data")
scenario_file = os.path.join(outfolder, f"params_scenarios.pickle")
with open(scenario_file, "rb") as fp:
    scenario_dict_list = pickle.load(fp)
    
scenario_dict_list[0].keys()

dict_keys(['token_scenario', 'usage_scenario', 'staking_scenario', 'params_dict', 'data_dict_list'])

## 3. Set ranges for tokenomic params

In [4]:
param_ranges_dict = {
    "tipping_rate": [0.2, 0.3, 0.4],
    "protocol_fee_rate": [1],
    "min_stake_amount": [2500],
    "min_stake_duration": [7*4],
    "validator_reward_share": [0.7, 0.8, 0.9],
    "staking_rewards_vesting_decay_rate": [np.log(2) / (i * 365) for i in [2.0, 4.0, 6.0]],
    "release_rate_a": [0.8, 1.0],
    "release_rate_b": [0.5],
    "release_rate_max":[0.0008, 0.0006],
    "ecosystem_refresh_size": [390_000_000],
    "staking_rewards_fund_size": [150_000_000],
}

## 3. Run parameter sweep

In [5]:
for sim, scenario_dict in enumerate(scenario_dict_list):
    params_dict = scenario_dict["params_dict"]
    data_dict_list = scenario_dict["data_dict_list"]
    sweep_df = run_param_sweep_sim(
        forecast_length,
        params_dict,
        param_ranges_dict,
        data_dict_list=data_dict_list,
    )
    sweep_df['token_scenario'] = scenario_dict['token_scenario']
    sweep_df['usage_scenario'] = scenario_dict['usage_scenario']
    sweep_df['staking_scenario'] = scenario_dict['staking_scenario']
    file_name = f"sim_df_scenario_{sim}.pickle"
    scenario_file = os.path.join(outfolder, file_name)
    with open(scenario_file, "wb") as fp:
        pickle.dump(sweep_df, fp)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:18<00:00,  5.71it/s]
